In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !cp "/content/gdrive/MyDrive/DS_Assignment/Test.zip" "/content/"
# !unzip "/content/Test.zip"
# !rm -rf "/content/Test.zip" 

# !cp "/content/gdrive/MyDrive/DS_Assignment/Trainning.zip" "/content/"
# !unzip "/content/Trainning.zip"
# !rm -rf "/content/Trainning.zip"

# !cp "/content/gdrive/MyDrive/DS_Assignment/Training_refined.zip" "/content/"
# !unzip "/content/Training_refined.zip"
# !rm -rf "/content/Training_refined.zip"

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import time
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Activation, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.applications import Xception

# from tensorflow.keras.applications import InceptionV3

In [6]:
# train_dir = "/content/Trainning/"
train_dir = "/content/Training_refined/"
test_dir = "/content/Test/"

IMG_SIZE = 254
NB_CHANNELS = 3
BATCH_SIZE = 8
NB_TRAIN_IMG = 39375
NB_VALID_IMG = 8617

In [ ]:
# lower = [0, 100, 100]
# upper = [35, 255, 255]
# lower = np.array(lower, dtype="uint8")
# upper = np.array(upper, dtype="uint8")

# def display(imgage, img_name):
#     plt.figure(figsize=(15,10))
#     plt.imshow(imgage)                                                                       
#     plt.title(img_name)
#     plt.show()
    
# def hsv2gray(input_path, flag=False):
#     img_bgr = cv2.imread(input_path)
#     img_blur = cv2.GaussianBlur(img_bgr.copy(), (1,1), 10)
#     img_hsv = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)
    
#     if flag:
#         img_hsv[:,:,2] += 50

#     mask = cv2.inRange(img_hsv, lower, upper)

#     if np.any(mask):
#         return 1
#     return 0

In [ ]:
# train_fire = os.listdir(train_dir+"Fire/")
# train_nofire = os.listdir(train_dir+"No_Fire/")
# file_path= []
# types =[]

# for i in train_fire:
#     inp_path = train_dir+"Fire/"+i
#     if (hsv2gray(inp_path, flag=True)):
#         file_path.append(inp_path)
#         types.append('Fire')

# for i in train_no_fire:
#     inp_path = train_dir+"No_Fire/"+i
#     if ~hsv2gray(inp_path):
#         file_path.append(inp_path)
#         types.append('No_Fire')

In [ ]:
# df = pd.DataFrame(data={'file_path':file_path, 'type':types})

In [ ]:
# os.mkdir("Training_refined")
# os.mkdir(os.path.join("Training_refined","Fire"))
# os.mkdir(os.path.join("Training_refined","No_Fire"))

In [ ]:
# import shutil
# for i in (df['file_path'].values):
#     if 'No_Fire' in i:
#         shutil.copy(i, os.path.join("Training_refined","No_Fire"))
#     else:
#         shutil.copy(i, os.path.join("Training_refined","Fire"))

In [8]:
train_datagen = ImageDataGenerator(
                                   rescale=1.0/255.0, 
                                   rotation_range=90, 
                                   horizontal_flip=True, 
                                   vertical_flip=True,
                                   zoom_range=[0.8,1.3],
                                  #  brightness_range=[0.7, 1.3],
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   shear_range = 0.1,
                                   fill_mode="nearest"
                                   ,validation_split=0.000
                                  )
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                  batch_size=BATCH_SIZE,
                                  target_size=(IMG_SIZE,IMG_SIZE),
                                  color_mode="rgb",
                                  shuffle = True,
                                  class_mode='binary',
                                  subset='training'
                              )
validation_generator = train_datagen.flow_from_directory(train_dir, 
                                  batch_size=BATCH_SIZE,
                                  target_size=(IMG_SIZE,IMG_SIZE),
                                  color_mode="rgb",
                                  shuffle = True,
                                  class_mode='binary',
                                  subset='validation'                
                                )

test_generator = test_datagen.flow_from_directory(test_dir, 
                                  batch_size=BATCH_SIZE,
                                  target_size=(IMG_SIZE,IMG_SIZE),
                                  color_mode="rgb",
                                  shuffle = False,
                                  class_mode='binary'
                              )

Found 8617 images belonging to 2 classes.


In [9]:
test_generator.class_indices

{'Fire': 0, 'No_Fire': 1}

In [ ]:
# def plotimages(images_arr):
#     fix, axes = plt.subplots(2, 3, figsize=(15,10))
#     axes = axes.flatten()
#     for img, ax in zip(images_arr, axes):
#         img = img.astype(np.uint8)
#         ax.imshow(img)
#         ax.axis('off')
#     plt.tight_layout()
#     plt.show()

# imgs, labels = next(train_generator)
# plotimages(imgs)
# # print(labels)

In [ ]:
input = Input(shape=(IMG_SIZE,IMG_SIZE,3))
pretrained = Xception(include_top=False, weights='imagenet', input_tensor=input)
output = pretrained.layers[-1].output
output = keras.layers.Flatten()(output)
pretrained = Model(pretrained.input, output)
# pretrained.summary()

In [ ]:
last_layer = pretrained.layers[-1].output
x = Dropout(0.5, name='dp1')(last_layer)
x = Dense(2048, activation='relu', name='fc1')(x)
# x = Dense(512, activation='relu', name='fc2')(x)
# x = Dropout(0.3, name='dp2')(x)
output = Dense(1, activation='sigmoid', name='output')(x)
model = Model(input, output)

# model.summary()

In [ ]:
for layer in model.layers[:-150]:
    layer.trainable = False

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

count =0 
for layer in model.layers:
    if layer.trainable:
        count+=1
print(count)

150


In [ ]:
callbacks = [EarlyStopping(patience=10, verbose=1, monitor='val_accuracy', mode='max'),
             ModelCheckpoint(filepath='inception_17apr.h5', monitor='val_accuracy', verbose=1, save_best_only=True)]
history = model.fit(train_generator, 
                              epochs=100,
                              validation_data=test_generator, 
                              verbose=1,
                              callbacks=callbacks)

In [ ]:
# !cp "/content/inception_17apr.h5" "/content/gdrive/MyDrive/DS_Assignment/"

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [23]:
# 0: Fire, 1: No Fire

def fire_model(input_path):
    img_bgr = cv2.imread(input_path)
    img_blur = cv2.GaussianBlur(img_bgr.copy(), (1,1), 10)
    img_hsv = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)
    img_hsv[:,:,2] += 40
    mask = cv2.inRange(img_hsv, lower, upper)

    if np.any(mask):
        return 0
    return 1

def no_fire_model(input_path):
    img_bgr = cv2.imread(input_path)
    img_blur = cv2.GaussianBlur(img_bgr.copy(), (1,1), 10)
    img_hsv = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(img_hsv, lower, upper)

    if np.any(mask):
        return 0
    return 1

In [ ]:
lower = [0, 100, 100]
upper = [35, 255, 255]
lower = np.array(lower, dtype="uint8")
upper = np.array(upper, dtype="uint8")
test_dir = "/content/Test"
opencv_model = []
for i, inp_path in enumerate(test_generator.filenames):
  opencv_model.append(no_fire_model(os.path.join(test_dir, inp_path)))
  if i%2000 == 0:
    print(i)

In [10]:
model = keras.models.load_model('/content/gdrive/MyDrive/tl_best_model_86492.h5')

In [13]:
pred= (model.predict(test_generator, verbose=1) > 0.5).astype("int64")

1078/1078 [==============================] - 84s 48ms/step


In [14]:
labels=(test_generator.classes)
print (confusion_matrix(labels, pred.ravel()))
print(classification_report(labels, pred.ravel()))

[[4355  782]
 [ 382 3098]]
              precision    recall  f1-score   support

           0       0.92      0.85      0.88      5137
           1       0.80      0.89      0.84      3480

    accuracy                           0.86      8617
   macro avg       0.86      0.87      0.86      8617
weighted avg       0.87      0.86      0.87      8617



In [29]:
result_df = pd.DataFrame({'file_name':test_generator.filenames, 'transfer_model':pred.ravel() })

In [30]:
result_df['opencv_model'] = opencv_model

In [36]:
print(confusion_matrix(labels, result_df['opencv_model'].values))
print(classification_report(labels, result_df['opencv_model'].values))

[[3699 1438]
 [  99 3381]]
              precision    recall  f1-score   support

           0       0.97      0.72      0.83      5137
           1       0.70      0.97      0.81      3480

    accuracy                           0.82      8617
   macro avg       0.84      0.85      0.82      8617
weighted avg       0.86      0.82      0.82      8617



In [49]:
result_df['result'] = result_df['opencv_model'] * result_df['transfer_model']

In [ ]:
# result_df.to_csv('result.csv', index=False)

In [50]:
print(confusion_matrix(labels, result_df['result'].values))
print(classification_report(labels, result_df['result'].values))

[[4595  542]
 [ 457 3023]]
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      5137
           1       0.85      0.87      0.86      3480

    accuracy                           0.88      8617
   macro avg       0.88      0.88      0.88      8617
weighted avg       0.88      0.88      0.88      8617

